In [1]:
# data types and event attencance/ tabling attendance then ready for join and report prep


#...issues with fab and staff emails

In [2]:
# this is boiler plate for when we move stuff into SQL. I think it will need to be cleaned before SQL. 

import pandas as pd
import pyodbc
import time
import numpy as np
from datetime import datetime, date, time, timedelta

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',None)


In [3]:

asanaData_raw = pd.read_csv('2025_events_Planning.csv')

In [4]:
#????????what do I do about tabling location and event location....need to combine them

columns_to_rename = {

       'Due Date': 'Date', 
       'Set Up Time ( 00:00 am/pm)':"Set Up Time",
    'Start Time ( 00:00 am/pm)':"Start Time",
       'End Time ( 00:00 am/pm)':"End Time",
    'Location':'Event Location',
       'Event Zip Code':"Zip", 
       'Reason for Not Attending':"Reason for NA",
       'How many staff attended?':"# Staff Attended",
    'How many FAB members attended?':"# FAB Attended",
       'How many interns attended?':"# Interns Attended",
    'How many volunteers attended?':"# Volunteers Attended",
       'How many people attended this event?':"# of People at Event",
       'How did you get the total public attendance number?':"Method of # P@E"
}



In [5]:
asanaData_raw.rename(columns=columns_to_rename,inplace = True)

In [6]:
for i in range(len(asanaData_raw)):
    if asanaData_raw.loc[i, 'Category'] == 'Community Event':
        asanaData_raw.loc[i, 'Location'] = asanaData_raw.loc[i, "Event Location"]

    elif asanaData_raw.loc[i, 'Category'] == 'Partner Tabling':
        asanaData_raw.loc[i, 'Location'] = asanaData_raw.loc[i, 'Tabling Location']
 
    else:
        asanaData_raw.loc[i, 'Category'] = 'error'
        print(f"Row {i}: Unexpected value in 'Category': {asanaData_raw.loc[i, 'Category']}")

Row 0: Unexpected value in 'Category': error
Row 45: Unexpected value in 'Category': error
Row 113: Unexpected value in 'Category': error


In [7]:
asanaData_raw['Date'] = pd.to_datetime(asanaData_raw['Date'], errors='coerce').dt.date





In [8]:
# to get the names of the columns that need to be renamed
asanaData_raw.columns



Index(['Task ID', 'Created At', 'Completed At', 'Last Modified', 'Name',
       'Section/Column', 'Assignee', 'Assignee Email', 'Start Date', 'Date',
       'Tags', 'Notes', 'Projects', 'Parent task', 'Blocked By (Dependencies)',
       'Blocking (Dependencies)', 'Event Status', 'Category',
       'Est. # of Staff NEEDED', 'Staff Attending', '# of Possible FAB',
       'FAB Attending', 'Open to Interns', 'Intern(s) Attending',
       'Volunteers Needed', 'Volunteers Attending', 'Set Up Time',
       'Start Time', 'End Time', 'Event Location', 'Tabling Location',
       'Address', 'Zip', 'PN?', 'Special Notes', 'Social Media', 'City',
       'KDL Branch', 'GRPL Branch', 'Assignee Attendance', 'Reason for NA',
       'Est. Attendance', '# Staff Attended', '# FAB Attended',
       '# Interns Attended', '# Volunteers Attended', 'Method of # P@E',
       'Host(s)', 'Volunteer Email Addresses (a,b,c):', 'Location'],
      dtype='object')

In [9]:
Coumns_to_Remove = {     'Created At',
                         'Completed At', 'Last Modified', 
                         'Section/Column',
                         'Assignee Email',
                         'Start Date',
                         'Tags',
                         'Notes', 
                         'Blocked By (Dependencies)',
                         'Blocking (Dependencies)',
                         'Est. # of Staff NEEDED', 
                         '# of Possible FAB',
                         'Open to Interns',
                         'Volunteers Needed', 
                         'Special Notes',
                         'Assignee Attendance','Event Location', 'Tabling Location',
       }

asanaData_raw.drop(Coumns_to_Remove,inplace = True, axis = 1)

In [10]:
asanaData_raw = asanaData_raw.drop(index=0)

In [11]:
#creating an Event Id so that when I join on the survey data I can still sort by Occurance, some tablings have the same name every time
asanaData_raw['Event ID'] = ['E' + str(i + 1) for i in range(len(asanaData_raw))]
asanaData_raw['Event ID'] = asanaData_raw['Event ID'].astype(str)

In [12]:
asanaData_raw.to_csv('Local_asanaData.csv',index = False)    # this if for joining on SurveyMonkey




Columns_to_Remove2 = {   'Staff Attending',
 'FAB Attending',
 'Intern(s) Attending',
 'Volunteers Attending'
       }

asanaData_raw_single = asanaData_raw.drop(Columns_to_Remove2, axis = 1)


asanaData_raw_single.to_csv('Local_asanaData_single.csv',index = False) # this is for staffing counts in tableau


In [13]:

#making a table of dates for the events to correct for large multi day events like pride
include_keywords = ['Grand Rapids Pride','SHOES Program']

asana_dates = asanaData_raw[
    #taking out tablings
    (asanaData_raw['Category'] == 'Community Event') &
    # taking out historical data where there will not be any survey data to join on
    (pd.to_datetime(asanaData_raw['Date']) > pd.to_datetime('2025-06-01')) &
    # taking out a few communtiy events that have multiples
    (asanaData_raw['Name'].str.strip().isin(include_keywords))
][['Name', 'Date']].copy()

asana_dates['Name'].unique()
#asana_dates.head(20)
asana_dates.to_csv('asana_dates_clean.csv', index=False)

In [14]:
asana_dates.head(260)
#asana_dates['Name'].unique()

# See any rows containing the problematic phrases
#mask = asanaData_raw['Name'].str.lower().str.contains('lincoln park|community night', na=False)
#print(asanaData_raw[mask][['Name']])

,Name,Date
50,SHOES Program,2025-08-15
108,Grand Rapids Pride,2025-06-22


In [15]:
asanaData_raw

,Task ID,Name,Assignee,Date,Projects,Parent task,Event Status,Category,Staff Attending,FAB Attending,Intern(s) Attending,Volunteers Attending,Set Up Time,Start Time,End Time,Address,Zip,PN?,Social Media,City,KDL Branch,GRPL Branch,Reason for NA,Est. Attendance,# Staff Attended,# FAB Attended,# Interns Attended,# Volunteers Attended,Method of # P@E,Host(s),"Volunteer Email Addresses (a,b,c):",Location,Event ID
1,1211083885845575,Southeast Area Farmers Market,Brooke Aernouts,2025-10-18,2025 Events Planning,NaN,Pending Decision,Community Event,NaN,NaN,NaN,NaN,10:30 am,11 am,4 pm,1030 Bemis St SE,49506.0,Yes,Yes,Grand Rapids,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,Our Kitchen Table/Southeast Area Farmers Market,NaN,Joe Taylor Park,E1
2,1210688094347344,Southeast Area Farmers Market,Cyle Blitstein,2025-11-15,2025 Events Planning,NaN,Pending Decision,Community Event,NaN,NaN,NaN,NaN,TBD,11am,4pm,1030 Bemis SE,49506.0,Yes,Yes,Grand Rapids,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,Our Kitchen Table,NaN,Joe Taylor Park,E2
3,1211080659646082,Envisioning Our Early Learning Community,Brooke Aernouts,2025-08-23,"Community Events Calendar,2025 Events Planning",NaN,Registration Needed,Community Event,NaN,NaN,NaN,NaN,12:30 pm,1 pm,4 pm,704 W Fulton,49504.0,No,No,Grand Rapids,NaN,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN,Parents for Healthy Homes,NaN,The Other Way Ministries,E3
4,1210920847790333,March For Fatherhood,Cyle Blitstein,2025-09-13,"Community Events Calendar,2025 Events Planning",NaN,Registered & Preparing,Community Event,NaN,NaN,NaN,NaN,TBD,TBD,TBD,TBD,10000.0,Yes,Yes,TBD,NaN,NaN,NaN,125.0,NaN,NaN,NaN,NaN,NaN,All Things Fatherhood,NaN,TBD,E4
5,1210412596892654,Lowell Harvest Celebration,Susi Alonso,2025-10-11,"Community Events Calendar,2025 Events Planning",NaN,Registered & Preparing,Community Event,NaN,NaN,NaN,NaN,TBD,TBD,TBD,Downtown Lowell,49331.0,No,Yes,Lowell,NaN,NaN,NaN,2000.0,NaN,NaN,NaN,NaN,NaN,Lowell Chamber of Commerce,NaN,Downtown Lowell,E5
6,1210332294272477,"Sparta Books, Blocks, and Balls",Lianna Tagle,2025-09-20,"Community Events Calendar,2025 Events Planning",NaN,Registered & Preparing,Community Event,"ltagle@familyfutures.org, salonso@familyfuture...",moniquevasquez@email.grcc.edu,NaN,NaN,TBD,TBD,TBD,TBD,0.0,No,Yes,TBD,NaN,NaN,NaN,500.0,2.0,1.0,NaN,NaN,NaN,Great Start Collaborative,NaN,"Sparta, MI",E6
7,1210950143901705,Red Flannel Festival,Cyle Blitstein,2025-10-04,"Community Events Calendar,2025 Events Planning",NaN,Registered & Preparing,Community Event,NaN,NaN,NaN,NaN,TBD,TBD,TBD,18 S. MAIN ST,49319.0,No,Yes,Cedar Springs,NaN,NaN,NaN,5000.0,NaN,NaN,NaN,NaN,NaN,Red Flannel Festival,NaN,Downtown Cedar Springs,E7
8,1210479561441476,Kendra Scott Give Back Event,Marilyn Kehr,2025-12-14,"Community Events Calendar,2025 Events Planning",NaN,Registered & Preparing,Community Event,NaN,NaN,NaN,NaN,12:50pm,1:00pm,4:00pm,1944 Breton Rd SE,49506.0,No,Yes,Grand Rapids,NaN,NaN,NaN,30.0,NaN,NaN,NaN,NaN,NaN,Kendra Scott,NaN,Kendra Scott Jewlery,E8
9,1210832589210915,Community Resource Fair at GRPL,Cyle Blitstein,2025-09-26,"Community Events Calendar,2025 Events Planning",NaN,Registered & Preparing,Community Event,NaN,NaN,NaN,NaN,noon or noon thirty,1pm,5pm,111 Library St NE,49503.0,Yes,Yes,Grand Rapids,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,Grand Rapids Public Library,NaN,Grand Rapids Public Library Main Branch,E9
10,1210833506538653,Grand Rapids Plays,Cyle Blitstein,2025-09-20,"Community Events Calendar,2025 Events Planning",NaN,Registered & Preparing,Community Event,NaN,NaN,NaN,NaN,noon?,1:00pm,3:30pm,940 Baldwin St SE,49506.0,Yes,Yes,Grand Rapids,NaN,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN,Playground GR,NaN,Congress Elementary,E10


In [16]:
# add a numberical line for data eval? just an idea